In [1]:
import os
import sys
import re
import pandas as pd

In [2]:
pattern = r"(?<=output_).[^_]*(?=_)"
news_list = set()
files = [f"output/{f}" for f in os.listdir("output/") if f.endswith(".csv")]

for f in files:
    news_name = re.search(pattern, f)[0]
    news_list.add(news_name)
    print(f)
        

output/output_ABC7_0_gcp.csv
output/output_ABC7_1_gcp.csv
output/output_ABC7_2_gcp.csv
output/output_ABCNews_0_gcp.csv
output/output_ABCNews_0_nyu.csv
output/output_ABCNews_1_nyu.csv
output/output_ABCNews_2_nyu.csv
output/output_ABCNews_3_nyu.csv
output/output_ABCNews_4_nyu.csv
output/output_ABCNews_5_nyu.csv
output/output_ABCNews_full_nyu.csv
output/output_ABCNews_nyu.csv
output/output_AssociatedPress_0_gcp.csv
output/output_AssociatedPress_1_gcp.csv
output/output_AssociatedPress_2_gcp.csv
output/output_BBCNews_0_local.csv
output/output_BBCNews_0_nyu.csv
output/output_BBCNews_1_nyu.csv
output/output_BBCNews_2_nyu.csv
output/output_BBCNews_3_nyu.csv
output/output_BBCNews_4_nyu.csv
output/output_BBCNews_nyu.csv
output/output_BloombergTelevision_0_local.csv
output/output_BloombergTelevision_0_nyu.csv
output/output_BloombergTelevision_1_local.csv
output/output_BloombergTelevision_1_nyu.csv
output/output_BloombergTelevision_2_local.csv
output/output_BloombergTelevision_2_nyu.csv
output/out

In [3]:
print(len(news_list))
news_list = list(news_list)
news_list[0]

36


'Breibart'

In [4]:
all_df_list = []
for channel in news_list:
    channel_files = [f for f in files if f.startswith(f"output/output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title
0,ItopNgRQiuY,Breibart,âBlack Lives Matter Is a Jokeâ â Black D...
1,DKcqY5v9jfk,Breibart,"Sinjar, Iraq"
2,cYyNMSrWqmM,Breibart,Pro-Trump Girl Band Performs âKeep America G...
3,Iir6Kim6mgc,Breibart,Dr. Fauci to CNN's Jim Acosta: Sound Bites Lik...
4,rfXxkDrkQGM,Breibart,Assyrian Woman Urges Rep. Ilhan Omar's Support...
...,...,...,...
33427,SZTYyFNSL5o,TIME,Robots Working In Elder Care | TIME
33428,1E1VKzNoGvY,TIME,Michelle Yeoh | TIME 100 Talks
33429,OZaw-GyzsN4,TIME,North Korea's Kim Jong Un Will Meet With Putin...
33430,nhFx2jtcBn0,TIME,Malfunctioning Heater Sparked Reports Of Activ...


In [5]:
df.to_csv("news_videos_Aug2023.csv", index=False, encoding="utf-8")